# Outpost Credentials (BMC Discovery)

This notebook reproduces the `outpost_creds` report using the Tideway library only.
It maps Discovery Outposts to the credential UUIDs they use, and includes the credential label.

> **NOTE:** This may take a while if there are a large amount of SessionResults, due to the limitations of the API.

## Requirements

Uncomment to install dependencies if needed.

In [ ]:
# %pip install -q tideway pandas pyyaml

import os, sys
from pathlib import Path
from typing import Any, Dict, List
import pandas as pd
import yaml


## Select Appliance (optional)

If `config.yaml` has multiple appliances, set by name or index. Defaults to the first.

In [ ]:
APPLIANCE_NAME = None   # e.g., 'prod' or 'dev'
APPLIANCE_INDEX = 0     # integer index if not using name selection


## Configuration (from config.yaml)

Loads target, token/token_file, API version, and SSL flag; prepares output folder.

In [ ]:
def _find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'config.yaml').exists():
            return p
    return start.parent

repo_root = _find_repo_root(Path.cwd())
config_path = repo_root / 'config.yaml'
with open(config_path, 'r') as fh:
    cfg = yaml.safe_load(fh) or {}

apps = cfg.get('appliances') or []
selected = None
if isinstance(apps, list) and apps:
    if APPLIANCE_NAME:
        selected = next((a for a in apps if a.get('name') == APPLIANCE_NAME), None)
        if selected is None:
            raise ValueError(f"No appliance named '{APPLIANCE_NAME}' in config.yaml")
    else:
        try:
            selected = apps[int(APPLIANCE_INDEX)]
        except Exception:
            selected = apps[0]

target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
if not target:
    raise ValueError('config.yaml missing "target"')

token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
if not token and token_file:
    tf_path = Path(token_file)
    if not tf_path.is_absolute():
        tf_path = repo_root / tf_path
    with open(tf_path, 'r') as tf:
        token = tf.read().strip()
if not token:
    raise ValueError('API token not found in config.yaml (token or token_file)')

API_VERSION = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
VERIFY_SSL = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
output_dir = repo_root / f'output_{sanitized}'
output_dir.mkdir(parents=True, exist_ok=True)

print('Base Host     :', target)
print('API Version   :', API_VERSION)
print('Verify SSL    :', VERIFY_SSL)
print('Output folder :', output_dir)

# Prefer local Tideway package if present
local_tideway = repo_root / 'Tideway'
if local_tideway.exists():
    sys.path.insert(0, str(local_tideway))
import importlib
tideway = importlib.import_module('tideway')
API_VERSION_NUM = API_VERSION.lstrip('v')
app = tideway.appliance(target, token, api_version=API_VERSION_NUM, ssl_verify=VERIFY_SSL)
twsearch = app.data()
twcreds = app.credentials()
try:
    about = app.api_about
    print('Appliance reachable:', about.status_code)
except Exception as e:
    print('Warning: failed to contact appliance /api/about:', e)


## Helpers

Result normalization, bulk search, and credential label lookup.

In [ ]:
def list_table_to_json(table_like: List[List[Any]]):
    if not table_like or not isinstance(table_like, list):
        return []
    if not table_like or not isinstance(table_like[0], list):
        return []
    headers = table_like[0]
    rows = table_like[1:]
    out = []
    for r in rows:
        try:
            out.append(dict(zip(headers, r)))
        except Exception:
            continue
    return out

def to_rows(payload: Any):
    if isinstance(payload, list):
        if payload and isinstance(payload[0], list):
            return list_table_to_json(payload)
        if payload and isinstance(payload[0], dict):
            return payload
        return []
    if hasattr(payload, 'json'):
        try:
            js = payload.json()
        except Exception:
            return []
        if isinstance(js, list) and js and isinstance(js[0], list):
            return list_table_to_json(js)
        if isinstance(js, list) and js and isinstance(js[0], dict):
            return js
        if isinstance(js, dict) and 'headings' in js and 'results' in js:
            return list_table_to_json([js['headings']] + list(js.get('results') or []))
        return []
    if isinstance(payload, dict) and 'headings' in payload and 'results' in payload:
        return list_table_to_json([payload['headings']] + list(payload.get('results') or []))
    return []

def tw_search_all(search, query: str, limit: int = 500):
    resp = search.search({'query': query}, format='object', limit=limit)
    return to_rows(resp)

def get_vault_label_map(creds_ep) -> Dict[str,str]:
    try:
        items = creds_ep.get_vault_credentials.json()
    except Exception:
        items = []
    mapping: Dict[str,str] = {}
    for c in items or []:
        if not isinstance(c, dict):
            continue
        uuid = str(c.get('uuid') or '').split('/')[-1]
        if uuid:
            mapping[uuid] = c.get('label')
    return mapping


## Queries and Outposts

TWQL to retrieve (credential, outpost) pairs and a REST call to list outposts and URLs.

In [ ]:
qry_outpost_credentials = '''
search SessionResult
show credential, credential as 'uuid', outpost
'''

# Fetch outposts list
op_resp = app.get('/discovery/outposts?deleted=false')
try:
    op_list = op_resp.json()
except Exception:
    op_list = []
id_to_url = {}
for op in op_list or []:
    if not isinstance(op, dict):
        continue
    op_id = op.get('id') or op.get('outpost') or op.get('outpost_id') or op.get('uuid')
    if op_id:
        id_to_url[str(op_id)] = op.get('url')
pd.DataFrame(op_list).head(10)


## Run and Assemble

Build the (Outpost URL, Outpost Id, Credential UUID, Credential Label) rows.

In [ ]:
rows = tw_search_all(twsearch, qry_outpost_credentials)
print('Pairs:', len(rows))
df = pd.DataFrame(rows) if rows else pd.DataFrame(columns=['credential','uuid','outpost'])
df = df.drop_duplicates()
display(df.head(10))
label_map = get_vault_label_map(twcreds)
out_rows = []
for r in df.to_dict(orient='records'):
    uuid = (r.get('uuid') or r.get('credential'))
    opid = r.get('outpost')
    if not uuid or not opid:
        continue
    uuid_s = str(uuid).split('/')[-1]
    url = id_to_url.get(str(opid))
    out_rows.append([url, str(opid), uuid_s, label_map.get(uuid_s)])
out_df = pd.DataFrame(out_rows, columns=['Outpost','Outpost Id','Credential UUID','Credential Label'])
out_df.head(10)


## Save to CSV

Writes `outpost_creds.csv` to the standard output folder.

In [ ]:
OUTPUT_CSV = str(output_dir / 'outpost_creds.csv')
out_df.to_csv(OUTPUT_CSV, index=False)
print(f'Saved to {OUTPUT_CSV}')
